In [1]:
import os

In [2]:
%pwd

'd:\\EndtoEndProject\\Chicken-Diesease-Classification-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd #U got into direct root directory so now i can run evrything.

'd:\\EndtoEndProject\\Chicken-Diesease-Classification-Project'

In [5]:
#Now we need to update the entity
#Entity is ntg but its a return type of a function. Let's say u r creating one function and wht shd be the return type of that function as I already mentioned like "ConfigBox" def load_json(path: Path) -> ConfigBox:
#U can also create custom return type using entity for this i will create one data class
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
#I need to read the YAML file so first of all i need to mention inside constants
from cnnClassifier.constants import * #It will import evrything watever i have inside constants
#Now i also read yaml to create directeries
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH, #It is called from constants
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root]) #u can directly call .artifacts_root instead of config['artifacts_root'] from config coz we mentioned the decorator"@ensure_annotations" (artifacts_root: artifacts)->key:value pair


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [9]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [25]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [26]:
#Lets create the pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-03-23 11:52:06,673: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-23 11:52:06,689: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-23 11:52:06,694: INFO: common: created directory at: artifacts]
[2024-03-23 11:52:06,698: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-23 11:52:11,927: INFO: 4037744573: artifacts/data_ingestion/Chicken-fecal-images.zip download! with following info: 
Connection: close
Content-Length: 11605269
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "6467b40274c03fc572e9e8b27ec308a3ac851f0f00a013d9af98e29782da7897"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2122:3CD25:1B4003:23EBE3:65FE750F
Accept-Ranges: bytes
Date: Sat, 23 Mar 2024 06:22:09 GMT
Via: 1.1 varnish
X-Served-By: cache-maa1023

In [ ]:
#Now i need to convert this into modular coding and lets see it is working or not.